In [2]:
# modules

import csv
import sqlite3
from collections import OrderedDict, Counter
import requests
import io
import os
import pandas as pd
import sys

In [129]:
# obtaining & parsing scopus data
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

import_path = "papers\\2018.csv"

counter_n = 0
counter_t = 0
papers = []
with open(import_path, 'r', encoding = 'UTF-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for item in row:
            if '\n' in row[item]:
                counter_n += 1
            if '\t' in row[item]:
                counter_t += 1
        papers.append(row)
df = pd.DataFrame(papers)

In [130]:
for cnt, paper in enumerate(papers):
    if len(list(paper.keys())) != 44:
        print(cnt)

In [3]:
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

db_name = 'papers.db'
papers_extension = 'csv'
papers_directory = 'papers'

for cnt, file in enumerate(os.listdir(papers_directory)):
    if file.split('.')[1] == papers_extension:
        papers = []
        with open(os.path.join(papers_directory, file), 'r', encoding = 'UTF-8-sig') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                papers.append(row)
            if cnt == 0:
                db = sqlite3.connect(db_name)
                cursor = db.cursor()
                db_headers = ', '.join(map(lambda item: '[' + item + ']', papers[0].keys()))
                cursor.execute(f'''
                    CREATE TABLE papers ({db_headers});
                ''')
                db.commit()
            
            cursor.executemany(f'''
                INSERT INTO papers ({db_headers}) 
                values (
                    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                    ?, ?, ?, ?
                );
            ''', [tuple(paper.values()) for paper in papers])
            db.commit()
db.close()

In [4]:
def dict_factory(cursor, row):
    d = OrderedDict()
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

db_name = 'papers.db'
papers_extension = 'csv'
papers_directory = 'papers'

db = sqlite3.connect(db_name)
db.row_factory = dict_factory
cursor = db.cursor()
cursor.execute('''SELECT * FROM papers WHERE Year=?''', ('2018',))
pp = cursor.fetchall()
len(pp)
db.close()

In [7]:
type(db) == sqlite3.Connection

True